# SIADS 516: Homework 3
Version 2.0.20201020.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

This homework assignment builds on the Spark DataFrame material we covered in class.
You will be using a compressed version of the Yelp Academic Dataset.  The data set is provided for you in the data/yelp-academic sub-folder of this notebook's directory and you should not need to download it again if you're working on the Coursera hosted notebook environment.

You might want to refer to the lecture companion notebooks (in workspace-files/resources/lecture_notebooks or equivalently via Coursera as "Ungraded Lab: Spark Core Demo" and "Ungraded Lab: Spark SQL Demo) for hints about libraries to import, how to set up a SparkSession, and how to read data files.

You will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

**You should take care to document your work, preferably using markdown blocks. In-code commenting is also 
a good idea.**

### <font color="magenta">Q1: How many users have received more than 5000 cool votes?</font>

In [4]:
# insert your code here
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('SIADS 516 Homework 3') \
    .getOrCreate() 

sc = spark.sparkContext

business = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('../non_auto_assignments/data/yelp_academic/yelp_academic_dataset_user.json.gz')

In [32]:
# import other required packages
from pyspark.sql.functions import explode, udf
from pyspark.sql.types import *
from pyspark.ml.feature import * 
import re
import string

In [13]:
# showing the schema from user DataFrame
user.printSchema()
# assign condition to extract rows that users received more than 5000 'cool' compliments
res1 = user.filter(user['compliment_cool'] > 5000).collect()

root
 |-- average_stars: double (nullable = true)
 |-- compliment_cool: long (nullable = true)
 |-- compliment_cute: long (nullable = true)
 |-- compliment_funny: long (nullable = true)
 |-- compliment_hot: long (nullable = true)
 |-- compliment_list: long (nullable = true)
 |-- compliment_more: long (nullable = true)
 |-- compliment_note: long (nullable = true)
 |-- compliment_photos: long (nullable = true)
 |-- compliment_plain: long (nullable = true)
 |-- compliment_profile: long (nullable = true)
 |-- compliment_writer: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- elite: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- friends: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- yelping_since: string (nullable = true)



In [14]:
len(res1)

79

There are 79 users who received more than 5000 'cool' compliments.

### <font color="magenta">Q2: What are the top 10 most useful positive reviews?</font>

In [15]:
# Preview schema of review DataFrame
review.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [16]:
# Preview data of the DataFrame
review.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|ujmEBvifdJM6h6RLv...|   0|2013-05-07 04:34:36|    1|Q1sbwvVQXV2734tPg...|  1.0|Total bill for th...|     6|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|   0|2017-01-14 21:30:33|    0|GJXCdrto3ASJOqKeV...|  5.0|I *adore* Travis ...|     0|yXQM5uF2jS6es16SJ...|
|WTqjgwHlXbSFevF32...|   0|2016-11-09 20:09:03|    0|2TzJjDVDEuAW6MR5V...|  5.0|I have to say tha...|     3|n6-Gk65cPZL6Uz8qR...|
|ikCg8xy5JIg_NGPx-...|   0|2018-01-09 20:56:38|    0|yi0R0Ugj_xUx_Nek0...|  5.0|Went in for a lun...|     0|dacAIZ6fTM6mqwW5u...|
|b1b1eb3uo-w561D0Z...|   0|2018-01-30 23:07:38|    0|11a8sVPMUFtaC7_AB...|  1.0|Today was 

In [17]:
# Order by descending useful count
review_desc = review.orderBy('useful', ascending=False)

In [26]:
# Display the top 10 reviews' text, review_id, useful counts
res2 = review_desc.select('review_id', 'text', 'useful' ).limit(10)

In [36]:
res2.show()

+--------------------+--------------------+------+
|           review_id|                text|useful|
+--------------------+--------------------+------+
|4ZN5ZWVoGd8er9giA...|Dinner for 1.

- ...|  1241|
|A8mLBytNM2zmjHgSp...|In retrospect, I ...|  1122|
|ZmEtySx0W_RSv07aY...|This restaurant i...|   970|
|EYbuFrEnVkVdavuRm...|This is the secon...|   846|
|O1YX1g7Wbf0rmcoud...|I actually suspec...|   808|
|aTLCBP_6lrCs2Qo3k...|"scary that peopl...|   781|
|EluaWiwRr0VhTwJe9...|I really wanted t...|   694|
|oo7W2IgyMMiraml-s...|A few years ago a...|   668|
|QovoEUcs34yvCCm5u...|I'd put zero star...|   650|
|69suGHxR3PVxicAgm...|We were very disa...|   578|
+--------------------+--------------------+------+



In [34]:
#corresponding review ids of the top 10 useful reviews
res2_ids_l = res2.select('review_id').collect()
#corresponding text of the top 10 useful reviews
res2_text_l = res2.select('text').collect()

The top 10 'most useful' reviews can be found in the spark dataframe res2, with corresponding information. pure text can be found in python list res2_text_l; review id can be found in python list res2_ids_l.

### <font color="magenta">Q3: During what hour of the day do most checkins occur?</font>

In [43]:
# Preview checkin DataFrame schema
checkin.printSchema()


root
 |-- business_id: string (nullable = true)
 |-- date: string (nullable = true)



Row(date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')

In [45]:
# Preview checkin DataFrame entry
checkin.select('date').first()

Row(date='2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016-10-15 02:45:18, 2016-11-18 01:54:50, 2017-04-20 18:39:06, 2017-05-03 17:58:02')

In [2]:
# Explode the checkin dataframe

datesplit = udf(lambda x: x.split(','), ArrayType(StringType()))

checkin_exploded = checkin.select(datesplit('date').alias('date_list')).withColumn('checkin_record', explode('date_list'))

In [9]:
checkin_exploded.show()

+--------------------+--------------------+
|           date_list|      checkin_record|
+--------------------+--------------------+
|[2016-04-26 19:49...| 2016-04-26 19:49:16|
|[2016-04-26 19:49...| 2016-08-30 18:36:57|
|[2016-04-26 19:49...| 2016-10-15 02:45:18|
|[2016-04-26 19:49...| 2016-11-18 01:54:50|
|[2016-04-26 19:49...| 2017-04-20 18:39:06|
|[2016-04-26 19:49...| 2017-05-03 17:58:02|
|[2011-06-04 18:22...| 2011-06-04 18:22:23|
|[2011-06-04 18:22...| 2011-07-23 23:51:33|
|[2011-06-04 18:22...| 2012-04-15 01:07:50|
|[2011-06-04 18:22...| 2012-05-06 23:08:42|
|[2011-06-04 18:22...| 2012-06-08 22:43:12|
|[2011-06-04 18:22...| 2012-08-06 23:20:52|
|[2011-06-04 18:22...| 2012-08-19 18:30:44|
|[2011-06-04 18:22...| 2013-01-27 23:49:51|
|[2011-06-04 18:22...| 2013-03-01 01:22:29|
|[2011-06-04 18:22...| 2013-03-23 21:53:47|
|[2011-06-04 18:22...| 2013-03-24 01:11:51|
|[2011-06-04 18:22...| 2013-05-20 00:12:25|
|[2011-06-04 18:22...| 2013-06-29 22:50:57|
|[2011-06-04 18:22...| 2013-07-0

In [19]:
# Extract hour from checkin Dataframe

get_hour = udf(lambda x: re.findall(r'\s(\d\d):', x)[0], StringType())
checkin_exploded.select(get_hour('checkin_record')).show()

+------------------------+
|<lambda>(checkin_record)|
+------------------------+
|                      19|
|                      18|
|                      02|
|                      01|
|                      18|
|                      17|
|                      18|
|                      23|
|                      01|
|                      23|
|                      22|
|                      23|
|                      18|
|                      23|
|                      01|
|                      21|
|                      01|
|                      00|
|                      22|
|                      15|
+------------------------+
only showing top 20 rows



In [30]:
res3 = checkin_exploded.select(get_hour('checkin_record').alias('hours'))

In [34]:
# Group by hours and count frequency for each hour
res3.groupBy('hours').count().collect()

[Row(hours='07', count=231417),
 Row(hours='15', count=617830),
 Row(hours='11', count=111769),
 Row(hours='01', count=1561788),
 Row(hours='22', count=1257437),
 Row(hours='16', count=852076),
 Row(hours='18', count=1272108),
 Row(hours='00', count=1491176),
 Row(hours='17', count=1006102),
 Row(hours='09', count=100568),
 Row(hours='05', count=485129),
 Row(hours='19', count=1502271),
 Row(hours='23', count=1344117),
 Row(hours='08', count=151065),
 Row(hours='03', count=1078939),
 Row(hours='02', count=1411255),
 Row(hours='06', count=321764),
 Row(hours='20', count=1350195),
 Row(hours='10', count=88486),
 Row(hours='12', count=178910),
 Row(hours='04', count=747453),
 Row(hours='13', count=270145),
 Row(hours='21', count=1238808),
 Row(hours='14', count=418340)]

Hour-wise, most checkins occur at 0100.

### <font color="magenta">Q4: Sentiment analysis</font>

a. List the 50 most common non-stopword words that are unique to *positive* reviews.
b. List the 50 most common non-stopword words that are unique to *negative* reviews.

You can use the stopword list that was introduced in the lecture materials or you can 
find/devise one of your own.

You will need to define what constitutes a positive review and what constitutes a negative review.  We highly recommend that you use the number of stars to figure this out.  Be sure to provide a rationale for your choice

As an example, consider the following two reviews:

* Positive: The meal was great, and the service was the best we ever experienced.
* Negative: The meal was awful.  It was the worst thing we ever experienced.

Assume our stopwords are {'the','was','and','the','was','we','it'}

* Positive unique: {'great', 'service', 'best'}

* Negative unique: {'awful', 'worst', 'thing'}

In this example, each unique word occurs just once, so the concept of "top 50" doesn't make sense.  For your data, you'll need to count the number of times each unique word occurs.

a. List the 50 most common non-stopword words that are unique to positive reviews

In [5]:
#According to the yelp site- write a review page, select 3 stars means 'ok', while 4 stars mean 'good' and 5 stars mean 'great'. Therefore we narrow down to reviews with 4-5 stars ratings as positive reviews. 
res4a_1 = review.filter(review['stars'] > 3.0)
res4a_1.select('text').show()

+--------------------+
|                text|
+--------------------+
|I *adore* Travis ...|
|I have to say tha...|
|Went in for a lun...|
|I'll be the first...|
|Like walking back...|
|Wow. So surprised...|
|Michael from Red ...|
|You can't really ...|
|Great lunch today...|
|We've been a huge...|
|Our family LOVES ...|
|If you are lookin...|
|The food is alway...|
|Pick any meat on ...|
|Great food, great...|
|PlumbSmart provid...|
|their pettuccine ...|
|ended up here bec...|
|Best chinese rest...|
|This place epitom...|
+--------------------+
only showing top 20 rows



In [20]:
# Create stopword transformer from pyspark.ml as stopword filtering function
Remover_res4 = StopWordsRemover(inputCol="words", outputCol="filtered" )


In [67]:
# Create lists of words, filtered with stop words, remove punctuations and clean up/ standardize
textsplit = udf(lambda x: re.sub(r'[^\w\s]', '', x).lower().split(), ArrayType(StringType()))
res4a_2 = res4a_1.select(textsplit('text').alias('words'))
res4a_3 = Remover_res4.transform(res4a_2)                 
res4a_3.show()


+--------------------+--------------------+
|               words|            filtered|
+--------------------+--------------------+
|[i, adore, travis...|[adore, travis, h...|
|[i, have, to, say...|[say, office, rea...|
|[went, in, for, a...|[went, lunch, ste...|
|[ill, be, the, fi...|[ill, first, admi...|
|[like, walking, b...|[like, walking, b...|
|[wow, so, surpris...|[wow, surprised, ...|
|[michael, from, r...|[michael, red, ca...|
|[you, cant, reall...|[cant, really, fi...|
|[great, lunch, to...|[great, lunch, to...|
|[weve, been, a, h...|[weve, huge, slim...|
|[our, family, lov...|[family, loves, f...|
|[if, you, are, lo...|[looking, best, p...|
|[the, food, is, a...|[food, always, go...|
|[pick, any, meat,...|[pick, meat, plan...|
|[great, food, gre...|[great, food, gre...|
|[plumbsmart, prov...|[plumbsmart, prov...|
|[their, pettuccin...|[pettuccine, fres...|
|[ended, up, here,...|[ended, raku, clo...|
|[best, chinese, r...|[best, chinese, r...|
|[this, place, epi...|[place, ep

In [68]:
# Return a list based on top counts from the words
res4a_4 = res4a_3.select('filtered').withColumn('word', explode('filtered'))\
.groupBy('word')\
.count()\
.sort('count', ascending=False)\
.take(50)
# View the list of most non-stop words related to positive reviews
res4a_4

[Row(word='great', count=2403454),
 Row(word='place', count=2113929),
 Row(word='good', count=2099933),
 Row(word='food', count=1969159),
 Row(word='service', count=1413577),
 Row(word='time', count=1350757),
 Row(word='like', count=1275360),
 Row(word='one', count=1199603),
 Row(word='get', count=1194370),
 Row(word='back', count=1107119),
 Row(word='really', count=1069876),
 Row(word='go', count=1018052),
 Row(word='also', count=989381),
 Row(word='best', count=947167),
 Row(word='love', count=891038),
 Row(word='always', count=854365),
 Row(word='nice', count=841867),
 Row(word='well', count=806333),
 Row(word='friendly', count=797308),
 Row(word='amazing', count=775425),
 Row(word='definitely', count=754587),
 Row(word='staff', count=737547),
 Row(word='ive', count=735655),
 Row(word='delicious', count=722219),
 Row(word='got', count=705668),
 Row(word='us', count=690800),
 Row(word='im', count=673208),
 Row(word='try', count=643972),
 Row(word='even', count=640415),
 Row(word='lit

In [69]:
# Clean up: answer for 4(a)
res4a = []
for i in res4a_4:
    w = re.findall(r"'([^']*)'", str(i))[0]
    res4a.append(w)
print(res4a)

['great', 'place', 'good', 'food', 'service', 'time', 'like', 'one', 'get', 'back', 'really', 'go', 'also', 'best', 'love', 'always', 'nice', 'well', 'friendly', 'amazing', 'definitely', 'staff', 'ive', 'delicious', 'got', 'us', 'im', 'try', 'even', 'little', 'dont', 'first', 'recommend', 'chicken', 'restaurant', 'come', 'came', 'made', 'menu', 'ordered', 'experience', 'much', 'went', 'make', 'everything', 'new', 'fresh', 'people', 'order', 'right']


b. List the 50 most common non-stopword words that are unique to negative reviews.

Insert your interpretation here.

In [72]:
#According to the yelp site- write a review page, select 3 stars means 'ok', while 4 stars mean 'good' and 5 stars mean 'great'. Therefore we narrow down to reviews with "below 3 stars" ratings as negative reviews. 
res4b_1 = review.filter(review['stars'] < 3.0)
res4b_1.select('text').show()

+--------------------+
|                text|
+--------------------+
|Total bill for th...|
|Today was my seco...|
|This place has go...|
|I was really look...|
|Walked in around ...|
|I cannot believe ...|
|Unfortunately, I ...|
|if i can give thi...|
|This review is in...|
|I tried this plac...|
|Love this place d...|
|Came here on a Th...|
|Went here last we...|
|They keep there a...|
|Met a friend for ...|
|We had dinner at ...|
|I am years out fr...|
|I've never experi...|
|Took my kids here...|
|Sadly this place ...|
+--------------------+
only showing top 20 rows



In [73]:
# Create lists of words, filtered with stop words, remove punctuations and clean up/ standardize
res4b_2 = res4b_1.select(textsplit('text').alias('words'))
res4b_3 = Remover_res4.transform(res4b_2)                 
res4b_3.show()

+--------------------+--------------------+
|               words|            filtered|
+--------------------+--------------------+
|[total, bill, for...|[total, bill, hor...|
|[today, was, my, ...|[today, second, t...|
|[this, place, has...|[place, gone, hil...|
|[i, was, really, ...|[really, looking,...|
|[walked, in, arou...|[walked, around, ...|
|[i, cannot, belie...|[believe, things,...|
|[unfortunately, i...|[unfortunately, m...|
|[if, i, can, give...|[give, place, sta...|
|[this, review, is...|[review, regards,...|
|[i, tried, this, ...|[tried, place, gi...|
|[love, this, plac...|[love, place, dow...|
|[came, here, on, ...|[came, thursday, ...|
|[went, here, last...|[went, last, week...|
|[they, keep, ther...|[keep, appointmen...|
|[met, a, friend, ...|[met, friend, din...|
|[we, had, dinner,...|[dinner, bellagio...|
|[i, am, years, ou...|[years, surgery, ...|
|[ive, never, expe...|[ive, never, expe...|
|[took, my, kids, ...|[took, kids, hang...|
|[sadly, this, pla...|[sadly, pl

In [74]:
# Return a list based on top counts from the words
res4b_4 = res4b_3.select('filtered').withColumn('word', explode('filtered'))\
.groupBy('word')\
.count()\
.sort('count', ascending=False)\
.take(50)
# View the list of most non-stop words related to negative reviews
res4b_4

[Row(word='food', count=867937),
 Row(word='get', count=744415),
 Row(word='place', count=731189),
 Row(word='service', count=718773),
 Row(word='one', count=710177),
 Row(word='like', count=697995),
 Row(word='time', count=696904),
 Row(word='back', count=684846),
 Row(word='good', count=557466),
 Row(word='us', count=549496),
 Row(word='go', count=521938),
 Row(word='even', count=515453),
 Row(word='dont', count=502146),
 Row(word='said', count=487167),
 Row(word='never', count=471824),
 Row(word='didnt', count=465724),
 Row(word='told', count=449468),
 Row(word='got', count=436888),
 Row(word='order', count=396105),
 Row(word='came', count=372923),
 Row(word='asked', count=365851),
 Row(word='went', count=361005),
 Row(word='really', count=356375),
 Row(word='ordered', count=349025),
 Row(word='minutes', count=339839),
 Row(word='im', count=330615),
 Row(word='people', count=326834),
 Row(word='first', count=313230),
 Row(word='going', count=290470),
 Row(word='know', count=288722),

In [75]:
# Clean up: answer for 4(b)
res4b = []
for i in res4b_4:
    w = re.findall(r"'([^']*)'", str(i))[0]
    res4b.append(w)
print(res4b)

['food', 'get', 'place', 'service', 'one', 'like', 'time', 'back', 'good', 'us', 'go', 'even', 'dont', 'said', 'never', 'didnt', 'told', 'got', 'order', 'came', 'asked', 'went', 'really', 'ordered', 'minutes', 'im', 'people', 'first', 'going', 'know', 'also', '2', 'another', 'come', 'customer', 'bad', 'two', 'took', 'way', 'experience', 'better', 'well', 'much', 'restaurant', 'take', 'still', 'called', 'make', 'give', 'great']
